# Pattern similarity analyses

### Goals of this script
Compute correlations between specific events of interest within a subject.  

1. import labels (regressor files) for each run type (trimmed but not shifted)
    - study runs
    - familiarization runs
    - reward runs
    - postScenes
    - postFaces
2. shift labels to account for hemodynamic lag, concatenate run types, other cleanup
3. load voxel x TR matrix for ROI(s) of interest
4. extract patterns of interest and compute correlations: 
    - preA, postB (differentiation)
    - preB, vioX & preB, vioY (B prediction during vio)
    - postB, rewardA (5 repetitions, B prediction during reward)
    - postB, postX & postB, postY
5. Fisher z-transform r-values, average pairs within condition, compute difference between conditions, convert back to r
6. 2nd order correlations:
    - Correlate B prediction with differentiation for violation pairs
    - Correlate postB,postXY value with differentiation in violation condition
    - Correlate B prediction during reward learning with differentiation
7. Randomization analyses:
    - differentiation: scramble preApostB pairings, compute preApostB for scrambled pairs, Fisher z-transform, average pairs within condition, and compute different between conditions. Do this 1000x.
    - B prediction during vio: scramble preBavgXY pairings, compute B prediction for scrambled pairs, correlate shuffled B prediction with original differentiation score in violation condition. Do this 1000x.
    - For each variable, compute the z-score of the true value relative to the mean and SD of the permuted distribution. 
    - Save this subject's z-scores. 

## Define subject

In [ ]:
sub = 'sub-XXX'
version = 'v4' #refers to version of the hippocampal masks
binarization_thresh='50' #refers to hippocampal mask thresholding
mask_list=['left_CA2+3+DG_1.5mm', 'right_CA2+3+DG_1.5mm', 'bilateral_CA2+3+DG_1.5mm',
      'left_CA2+3_1.5mm', 'right_CA2+3_1.5mm', 'bilateral_CA2+3_1.5mm',
      'left_DG_1.5mm', 'right_DG_1.5mm', 'bilateral_DG_1.5mm',
      'left_CA1_1.5mm', 'right_CA1_1.5mm', 'bilateral_CA1_1.5mm']

## Import necessary packages

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import numpy as np
import nibabel as nib
import pandas as pd
import nilearn
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
from nilearn.masking import intersect_masks
from nilearn import image
from nilearn import plotting
from nilearn.plotting import plot_roi
from scipy import stats
import matplotlib.pyplot as plt 
import scipy.io
from mpi4py import MPI
import os
from os import path
import pickle 
import time
from scipy.sparse import random
from scipy.stats import zscore
from scipy.stats.stats import pearsonr   
from scipy.spatial.distance import euclidean
from pathlib import Path
from shutil import copyfile
import seaborn as sns
#import plotly.express as px

%matplotlib inline 
%autosave 5

## Check versioning

In [ ]:
from platform import python_version
print('The python version is {}.'.format(python_version()))
print('The numpy version is {}.'.format(np.__version__))
print('The nilearn version is {}.'.format(nilearn.__version__))
print('The seaborn version is {}.'.format(sns.__version__))

assert python_version()== '3.7.6'
assert nilearn.__version__=='0.6.2'

## Load settings

In [ ]:
# Set printing precision
np.set_printoptions(precision=2, suppress=True)

# load some helper functions
sys.path.insert(0, '/jukebox/norman/emcdevitt/studies/SVD/code/mainanalysis')
import svd_utils
#from svd_utils import load_svd_epi_data, shift_timing, label2TR, mask_data

# load some constants
from svd_utils import n_runs, run_names, svd_TR, TRs_run

# Parameters
svd_hrf_lag = svd_TR*3
n_trunc_beginning=9 #Number of volumes to trim from beginning of run
n_trunc_end=5 #Number of volumes to trim from end of run

# label dictionaries
conditions = {1:"vio", 2:"nonvio", 3:"vio-nonvio"}
groups = {0:"wake", 1:"NREM", 2:"REM"}
categories = {0:"rest", 1:"face", 2:"scene", 3:"object"}
scene_subcategories = {1:"indoor", 2:"outdoor"}
face_subcategories = {1:"female", 2:"male"}
pair_type = {1:"outdoor/outdoor", 2:"indoor/indoor", 3: "indoor/outdoor", 4: "outdoor/indoor"}
reward_cond = {1:"reward", 2:"neutral"}

In [ ]:
# Directories
svd_bids_dir = '/jukebox/norman/emcdevitt/studies/SVD/data/bids/Norman/McDevitt/7137_viodiff/' #fmri data
behavioral_dir = '/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/' #regressor files
out_dir='/jukebox/norman/emcdevitt/studies/SVD/data/mainanalysis/output/notebook-05-pattern-similarity_%s_thresh-%s/' % (version, binarization_thresh) #folder where outputs of this notebook will be saved for each subject

# make out_dir if it doesn't exist
if os.path.exists(out_dir)==False:
    print('making new directory: %s' %out_dir)
    print('')
    os.mkdir(out_dir) 

deriv_dir=svd_bids_dir + 'v1.2.3_derivatives/'
preproc_dir=deriv_dir + 'fmriprep/%s/' % sub
anat_dir=deriv_dir + 'deface/' #defaced T1w images
mask_fold_other=deriv_dir + 'firstlevel/%s/masks/' % sub
mask_fold_hipp=deriv_dir + 'firstlevel/%s/rois_ashs/t1space_%s/threshold-%s/' % (sub, version, binarization_thresh)
epi_data_version='masked_epi_data_%s/threshold-%s' % (version, binarization_thresh)
epi_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)

print('bids dir = %s' % (svd_bids_dir))
print('')
print('mask dir = %s' % (mask_fold_hipp))
print('')
print('epi data dir = %s' % (epi_dir))
print('')
print('output dir = %s' % (out_dir))
print('')
print('ROIs = %s' % (mask_list))
print('')
print('LIST OF ALL TASKS:', run_names)
print('')

## Before running for subjects individually, check that necessary files exist for all subjects
data used in this notebook: study-and-postscenes, postscenes, postfaces, familiarization, reward

In [ ]:
# subject_list = [104,105,106,107,108,109,110,111,112,114,115,116,117,119,120,122,125,126,127,129,132,133,134,135,137,139,140,141,
#                142,143,144,145,146,147,148,149,150,151,152,153,155,156,158,159,160,161,162,164,165,166,168,169,170,172,173,174,175,
#                176,177,178,179,180,181,182,184,185,186,187,188] 
# # excluded subjects: 123,128,130,136,167

# n_subjects = len(subject_list)
# print('n_subjects:', n_subjects)
# print('')

# def main(in_file,sub,mask):
#     if path.exists(in_file)==False: #only prints if file doesn't exist
#         print (sub+ ' '+mask+" file exists: "+str(path.exists(in_file)))

# print('checking that all study-and-postscenes %s files exist' %epi_data_version)
# for subject in range(0, n_subjects):
#     this_sub = int(subject_list[subject])
#     sub = 'sub-%s' % (this_sub)
#     sub_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)
#     for mask_counter in range(len(mask_list)):
#         this_mask = mask_list[mask_counter]
#         in_file = (sub_dir + '%s_task-study-and-postscenes_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
#         if __name__== "__main__":
#             main(in_file,sub,this_mask)
# print('')

# print('checking that all postscenes %s files exist' %epi_data_version)
# for subject in range(0, n_subjects):
#     this_sub = int(subject_list[subject])
#     sub = 'sub-%s' % (this_sub)
#     sub_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)
#     for mask_counter in range(len(mask_list)):
#         this_mask = mask_list[mask_counter]
#         in_file = (sub_dir + '%s_task-postscenes_run-01_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
#         if __name__== "__main__":
#             main(in_file,sub,this_mask)
# print('')

# print('checking that all postfaces %s files exist' %epi_data_version)
# for subject in range(0, n_subjects):
#     this_sub = int(subject_list[subject])
#     sub = 'sub-%s' % (this_sub)
#     sub_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)
#     for mask_counter in range(len(mask_list)):
#         this_mask = mask_list[mask_counter]
#         in_file = (sub_dir + '%s_task-postfaces_run-01_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
#         if __name__== "__main__":
#             main(in_file,sub,this_mask)
# print('')

# print('checking that all familiarization %s files exist' %epi_data_version)
# for subject in range(0, n_subjects):
#     this_sub = int(subject_list[subject])
#     sub = 'sub-%s' % (this_sub)
#     sub_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)
#     for mask_counter in range(len(mask_list)):
#         this_mask = mask_list[mask_counter]
#         in_file = (sub_dir + '%s_task-familiarization_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
#         if __name__== "__main__":
#             main(in_file,sub,this_mask)
# print('')

# print('checking that all reward %s files exist' %epi_data_version)
# for subject in range(0, n_subjects):
#     this_sub = int(subject_list[subject])
#     sub = 'sub-%s' % (this_sub)
#     sub_dir= deriv_dir + 'firstlevel/%s/%s/' % (sub,epi_data_version)
#     for mask_counter in range(len(mask_list)):
#         this_mask = mask_list[mask_counter]
#         in_file = (sub_dir + '%s_task-reward_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
#         if __name__== "__main__":
#             main(in_file,sub,this_mask)

## Load subject's T1w and masks

In [ ]:
# load subject's defaced T1 image (merged T1 from fmriprep)
t1_file = anat_dir + sub + '_desc-preproc_T1w_defaced.nii.gz'
t1_img = image.load_img(t1_file) 

# Make a function to load the mask data
def load_svd_mask(ROI_name, sub):
    """Load the mask for the svd data 
    Parameters
    ----------
    ROI_name: string
    sub: string 
    
    Return
    ----------
    the requested mask
    """    
    # load the mask
    if ROI_name == 'bilateral_oc-temp':
        mask_fold=mask_fold_other
    else:
        mask_fold=mask_fold_hipp
    maskfile = (mask_fold + sub + "_%s.nii.gz" % (ROI_name))
    mask = nib.load(maskfile)
    print("Loaded mask: %s" % (ROI_name))
    return mask

In [ ]:
# plot this subject's ROIs
for mask_counter in range(len(mask_list)):
        this_mask = mask_list[mask_counter]
        mask = load_svd_mask(mask_list[mask_counter], sub) # load the mask for the corresponding ROI
        
        # plot mask overlayed on subject's T1
        plot_roi(mask, bg_img=t1_img, title=this_mask)

## Study runs
### Prep labels, load BOLD data

In [ ]:
# Study runs
task = 'study'
ses = 'ses-01'
task_index = run_names.index(task)
n_runs_study = n_runs[task_index]
TRs_run_study=TRs_run[task_index]-n_trunc_beginning-n_trunc_end
print('TASK:', task)
print('task index:', task_index)
print('TR = %s seconds' % (svd_TR))
print('TRs per run before trimming = %s' % (TRs_run[task_index]))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('TRs per run after trimming = %s' % (TRs_run_study))
print('')

### Load study regressor file
/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/task-study_regRow_reshaped.mat

In [ ]:
study_row_labels={'cond': 0, 'repetition': 1, 'pairID_orig': 2, 'ipi': 3, 'order': 4, 'subcategory': 5,
                'category': 6, 'imgID': 7, 'learningBlk': 8, 'learningTrial': 9, 'time': 10,
                'response': 11, 'rt': 12, 'accuracy': 13, 'tr_original': 14, 'tr_trimmed': 15,
                'uniquePairID': 16, 'exposureNum': 17, 'index': 18, 'cond_orig': 19} #index and cond_orig rows to be added
print(study_row_labels)

In [ ]:
# load stimulus labels from regressor file for each run and concatenate
# NOTE: Regressor files are already trimmed (beginning only), but not shifted, in Matlab using gen_study_regressor_0101.m

stim_label = []
stim_label_allruns = []
stim_label_allruns_shifted = []

# take individual run regressor files and concatenate them
for run in range(1, n_runs_study + 1):
    in_file = (behavioral_dir + '%s_%s_task-%s_regressor-noshift-trim%dTRs-reshaped_run-0%d.mat' % (sub, ses, task, n_trunc_beginning, run))
    
    # Load in data from matlab and trim end of labels
    stim_label = scipy.io.loadmat(in_file);
    stim_label = np.array(stim_label['regressor']);
    stim_label = stim_label[:,:-n_trunc_end] # trim label end

    # Store the data
    if run == 1:
        stim_label_allruns = stim_label;
    else:       
        stim_label_allruns = np.hstack((stim_label_allruns, stim_label))

print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('')
print('Trimmed (but not shifted) labels should begin with 3-0-3-0 and end with 9 trailing zeros')
print(stim_label_allruns[0,:21])
print(stim_label_allruns[0,-20:])

# Plot the labels
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label_allruns[0,:], c='orange')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')

In [ ]:
# Shift the data labels to account for hemodynamic lag
shift_size = int(svd_hrf_lag / svd_TR)  # Convert the shift into TRs
print('shift by %s TRs' % (shift_size))

zero_shift = np.zeros((np.shape(stim_label_allruns)[0],shift_size)) #columns to insert at beginning
end_trim = np.shape(stim_label_allruns)[1]-shift_size #trim columns at the end

# insert shift columns at beginning and trim columns at end
stim_label_allruns_shifted = np.hstack((zero_shift,stim_label_allruns[:,0:end_trim])) 

# stim_label_allruns_shifted = shift_timing(stim_label_allruns[0,:], shift_size)
print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('stim_label_allruns_shifted has shape: ', np.shape(stim_label_allruns_shifted))
print('')
print('Trimmed AND shifted labels should have 3 leading zeros and 6 trailing zeros')
print(stim_label_allruns_shifted[0,:15])
print(stim_label_allruns_shifted[0,-15:])

# Plot the original and shifted labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label_allruns[0,:], label='original', c='orange')
ax.plot(stim_label_allruns_shifted[0,:], label='shifted', c='blue')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')
ax.legend()

In [ ]:
# add index row
index_list = np.arange(len(stim_label_allruns_shifted[1]))
stim_label_allruns_shifted=np.vstack([stim_label_allruns_shifted, index_list])

# copy original condition labels (prestudy=3) to bottom row
stim_label_allruns_shifted=np.vstack([stim_label_allruns_shifted, stim_label_allruns_shifted[0,:]])

print(stim_label_allruns_shifted[:,:10])

In [ ]:
# update condition label for prelearning scenes (originally all prelearning scenes were coded as '3')

print('uniquePairID, 1-48=cond1 and 49-96=cond2:')
print(stim_label_allruns_shifted[study_row_labels['uniquePairID'],:15]) # IDs 1-48=cond1, 49-96=cond2
print('')

def numberfunc(x):
    if x == 0:
        return 0
    elif x > 48:
        return 2
    else:
        return 1

recode_condition = np.array(list(map(numberfunc, stim_label_allruns_shifted[study_row_labels['uniquePairID'],])))
stim_label_allruns_shifted[study_row_labels['cond'],:] = recode_condition

print('original condition:', stim_label_allruns_shifted[study_row_labels['cond_orig'],:15])
print('recoded condition:', recode_condition[:15])

# Plot the vio/nonvio condition labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label_allruns_shifted[study_row_labels['cond_orig'],-50:], label='original', c='blue') # this row includes prelearning=3
ax.plot(stim_label_allruns_shifted[study_row_labels['cond'],-50:], label='recoded', c='orange') # this row codes for vio or nonvio
ax.set_ylabel('vio or nonvio')
ax.set_xlabel('TR')
ax.legend()


In [ ]:
# check that condition labels match for non-prelearning trials
for x in range(len(stim_label_allruns_shifted[1])):
    if stim_label_allruns_shifted[study_row_labels['cond_orig'],x] != 3:
        assert stim_label_allruns_shifted[study_row_labels['cond_orig'],x] == stim_label_allruns_shifted[study_row_labels['cond'],x]

In [ ]:
# rename labels
study_labels_shifted = stim_label_allruns_shifted
print('study label list - shape:', study_labels_shifted.shape)

### Load Study voxel x TR BOLD data:

In [ ]:
# load voxel x TR data for each ROI
masked_data = []
masked_data_all = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    # load the mask for the corresponding ROI
    this_mask = mask_list[mask_counter]
        
    # Load in data from matlab
    in_file = (epi_dir + '%s_task-study-and-postscenes_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
    masked_data = scipy.io.loadmat(in_file)
    masked_data = np.array(masked_data['data'])
    masked_data = masked_data[:,:-200] #remove postscenes timepoints
    print(this_mask, masked_data.shape)
    assert masked_data.shape[1] == study_labels_shifted.shape[1] #check that BOLD data and labels have same number of timepoints
    masked_data_all[mask_counter] = masked_data

In [ ]:
# check dimensionality of the data and plot value of voxel_id across timeseries; make sure data are z-scored 
num_voxels = [0] * len(mask_list)
voxel_id = 10
for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    num_voxels[mask_counter] = masked_data_all[mask_counter].shape[0] #save number of voxels in each mask
    
    f, ax = plt.subplots(1,1, figsize=(14,5))
    ax.plot(masked_data_all[mask_counter][voxel_id,:])
    ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (this_mask, voxel_id))
    ax.set_xlabel('TR')
    ax.set_ylabel('Voxel Intensity')

In [ ]:
# Reshape BOLD data and rename
bold_data=[]
study_bold_data = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    print(this_mask)
        
    # # Pull out the indexes
    #indexed_data = masked_data_all[mask_counter][:,label_index] #this pulls out columns associated with non-zero labels in the epi data
    
    # transpose bold data to make it timepoints x n_voxels
    bold_data = np.transpose(masked_data_all[mask_counter])
    print('Original BOLD data shape:', masked_data_all[mask_counter].shape)
    print('Transposed BOLD data shape:', bold_data.shape)
    print('')

    study_bold_data[mask_counter] = bold_data

## Postscenes run

In [ ]:
task = 'postScenes'
ses = 'ses-02'
task_index = run_names.index(task)
n_runs_postscenes = n_runs[task_index]
TRs_run_postscenes=TRs_run[task_index]-n_trunc_beginning-n_trunc_end
print('TASK:', task)
print('task index:', task_index)
print('TR = %s seconds' % (svd_TR))
print('TRs per run before trimming = %s' % (TRs_run[task_index]))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('TRs per run after trimming = %s' % (TRs_run_postscenes))
print('')

### Load postscenes regressor file
/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/task-postStudy_regRow_reshaped.mat

In [ ]:
postscenes_row_labels={'cond': 0, 'repetition': 1, 'pairID_orig': 2, 'ipi': 3, 'order': 4, 'subcategory': 5,
                'category': 6, 'imgID': 7, 'learningBlk': 8, 'learningTrial': 9, 'time': 10,
                'response': 11, 'rt': 12, 'accuracy': 13, 'tr_original': 14, 'tr_trimmed': 15,
                'uniquePairID': 16, 'exposureNum': 17, 'index': 18} #index row to be added
print(postscenes_row_labels)

In [ ]:
# NOTE: Regressor files are already trimmed (beginning only), but not shifted, in Matlab using gen_study_regressor_0101.m
stim_label = []
stim_label_shifted = []

in_file = (behavioral_dir + '%s_%s_task-%s_regressor-noshift-trim%dTRs-reshaped_run-01.mat' % (sub, ses, task, n_trunc_beginning))
stim_label = scipy.io.loadmat(in_file) # Load in data from matlab
stim_label = np.array(stim_label['regressor'])
stim_label = stim_label[:,:-n_trunc_end] # trim label end

print('stim_label has shape: ', np.shape(stim_label))
print('')
print('Trimmed (but not shifted) labels should begin with alternating 1s/2s and 0s and end with 9 trailing zeros')
print(stim_label[0,:21])
print(stim_label[0,-20:])

# Plot the labels
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label[0,:], c='orange')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')

In [ ]:
# Shift the data labels to account for hemodynamic lag
shift_size = int(svd_hrf_lag / svd_TR)  # Convert the shift into TRs
print('shift by %s TRs' % (shift_size))

zero_shift = np.zeros((np.shape(stim_label)[0],shift_size)) #columns to insert at beginning
end_trim = np.shape(stim_label)[1]-shift_size #trim columns at the end
# insert shift columns at beginning and trim columns at end
stim_label_shifted = np.hstack((zero_shift,stim_label[:,0:end_trim])) 

print('stim_label has shape: ', np.shape(stim_label))
print('stim_label_shifted has shape: ', np.shape(stim_label_shifted))
print('')
print('Trimmed AND shifted labels should have 3 leading zeros and 6 trailing zeros')
print(stim_label_shifted[0,:15])
print(stim_label_shifted[0,-15:])

# Plot the original and shifted labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label[0,:], label='original', c='orange')
ax.plot(stim_label_shifted[0,:], label='shifted', c='blue')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')
ax.legend()

In [ ]:
# add index row
index_list = np.arange(len(stim_label_shifted[1]))
stim_label_shifted=np.vstack([stim_label_shifted, index_list])
print(stim_label_shifted[:,:10])

In [ ]:
# rename labels
postscenes_labels_shifted = stim_label_shifted
print('label list - shape:', postscenes_labels_shifted.shape)

### Load Postscenes voxel x TR BOLD data:

In [ ]:
# load voxel x TR data for each ROI
masked_data = []
masked_data_all = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    # load the mask for the corresponding ROI
    this_mask = mask_list[mask_counter]
    # Load in data from matlab
    in_file = (epi_dir + '%s_task-postscenes_run-01_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
    masked_data = scipy.io.loadmat(in_file)
    masked_data = np.array(masked_data['data'])
    assert masked_data.shape[1] == postscenes_labels_shifted.shape[1] #check that BOLD data and labels have same number 
    print(this_mask, masked_data.shape)
    masked_data_all[mask_counter] = masked_data

In [ ]:
# check dimensionality of the data and plot value of voxel_id across timeseries; make sure data are z-scored 
num_voxels = [0] * len(mask_list)
voxel_id = 10
for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    num_voxels[mask_counter] = masked_data_all[mask_counter].shape[0] #save number of voxels in each mask
    
    f, ax = plt.subplots(1,1, figsize=(14,5))
    ax.plot(masked_data_all[mask_counter][voxel_id,:])
    ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (this_mask, voxel_id))
    ax.set_xlabel('TR')
    ax.set_ylabel('Voxel Intensity')

In [ ]:
# Reshape BOLD data and rename
bold_data=[]
postscenes_bold_data = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    print(this_mask)
    # transpose bold data to make it timepoints x n_voxels
    bold_data = np.transpose(masked_data_all[mask_counter])
    print('Original BOLD data shape:', masked_data_all[mask_counter].shape)
    print('Transposed BOLD data shape:', bold_data.shape)
    print('')

    postscenes_bold_data[mask_counter] = bold_data

## Familiarization runs

In [ ]:
task = 'familiarization'
ses = 'ses-02'
task_index = run_names.index(task)
n_runs_familiarization = n_runs[task_index]
TRs_run_familiarization=TRs_run[task_index]-n_trunc_beginning-n_trunc_end
print('TASK:', task)
print('task index:', task_index)
print('number of task runs:', n_runs_familiarization)
print('TR = %s seconds' % (svd_TR))
print('TRs per run before trimming = %s' % (TRs_run[task_index]))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('TRs per run after trimming = %s' % (TRs_run_familiarization))
print('expected length of BOLD data series:', TRs_run_familiarization*n_runs_familiarization)
print('')

### Load familiarization regressor file
row labels: '/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/task-familiarization_regRow_all.mat'

In [ ]:
fam_row_labels={'cond': 0, 'repetition': 1, 'pairID_orig': 2, 'ipi': 3, 'order': 4, 'subcategory': 5,
                'category': 6, 'imgID': 7, 'learningTrialOnset': 8, 'trialNum': 9, 'learningBlk': 10,
                'index': 11, 'pairtype': 12, 'cycle': 13, 'rewardcond': 14, 'time': 15,
                'response': 16, 'rt': 17, 'accuracy': 18, 'onsetOutcome': 19, 'recordedOutcome': 20,
                'fMRItriggerOutcome': 21, 'run': 22, 'uniquePairID': 23, 'tr_original': 24, 'tr_trimmed': 25}
print(fam_row_labels)

In [ ]:
# load stimulus labels from regressor file for each run and concatenate
# NOTE: Regressor files are already trimmed (beginning only), but not shifted, in Matlab using gen_reward_regressor_0101.m

stim_label = []
stim_label_allruns = []
stim_label_allruns_shifted = []

# take individual run regressor files and concatenate them
for run in range(1, n_runs_familiarization + 1):
    in_file = (behavioral_dir + '%s_%s_task-%s_regressor-noshift-trim%dTRs_run-0%d.mat' % (sub, ses, task, n_trunc_beginning, run))
    
    # Load in data from matlab and trim end of labels
    stim_label = scipy.io.loadmat(in_file);
    stim_label = np.array(stim_label['regressor']);
    stim_label = stim_label[:,:-n_trunc_end] # trim label end

    # Store the data
    if run == 1:
        stim_label_allruns = stim_label;
    else:       
        stim_label_allruns = np.hstack((stim_label_allruns, stim_label))

print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('')
print('Trimmed (but not shifted) labels should begin with X-0-0-X-0-0-X and end with 10 trailing zeros')
print(stim_label_allruns[0,:21])
print(stim_label_allruns[0,-20:])

# Plot the labels
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label_allruns[0,:], c='orange')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')

In [ ]:
# Shift the data labels to account for hemodynamic lag
shift_size = int(svd_hrf_lag / svd_TR)  # Convert the shift into TRs
print('shift by %s TRs' % (shift_size))

zero_shift = np.zeros((np.shape(stim_label_allruns)[0],shift_size)) #columns to insert at beginning
end_trim = np.shape(stim_label_allruns)[1]-shift_size #trim columns at the end

# insert shift columns at beginning and trim columns at end
stim_label_allruns_shifted = np.hstack((zero_shift,stim_label_allruns[:,0:end_trim])) 

print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('stim_label_allruns_shifted has shape: ', np.shape(stim_label_allruns_shifted))
print('')
print('Trimmed AND shifted labels should have 3 leading zeros and 7 trailing zeros')
print(stim_label_allruns_shifted[0,:15])
print(stim_label_allruns_shifted[0,-15:])

# Plot the original and shifted labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label_allruns[0,:], label='original', c='orange')
ax.plot(stim_label_allruns_shifted[0,:], label='shifted', c='blue')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')
ax.legend()

In [ ]:
# reset index row
index_list = np.arange(len(stim_label_allruns_shifted[1]))
stim_label_allruns_shifted[fam_row_labels['index'],:]=index_list

In [ ]:
# rename labels
familiarization_labels_shifted = stim_label_allruns_shifted
print('familiarization label list - shape:', familiarization_labels_shifted.shape)

### Load Familiarization voxel x TR BOLD data:

In [ ]:
# load voxel x TR data for each ROI
masked_data = []
masked_data_all = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    # load the mask for the corresponding ROI
    this_mask = mask_list[mask_counter]
    # Load in data from matlab
    in_file = (epi_dir + '%s_task-%s_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask))
    masked_data = scipy.io.loadmat(in_file)
    masked_data = np.array(masked_data['data'])
    assert masked_data.shape[1] == familiarization_labels_shifted.shape[1] #check that BOLD data and labels have same number 
    print(this_mask, masked_data.shape)
    masked_data_all[mask_counter] = masked_data

In [ ]:
# check dimensionality of the data and plot value of voxel_id across timeseries; make sure data are z-scored 
num_voxels = [0] * len(mask_list)
voxel_id = 10
for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    num_voxels[mask_counter] = masked_data_all[mask_counter].shape[0] #save number of voxels in each mask
    
    f, ax = plt.subplots(1,1, figsize=(14,5))
    ax.plot(masked_data_all[mask_counter][voxel_id,:])
    ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (this_mask, voxel_id))
    ax.set_xlabel('TR')
    ax.set_ylabel('Voxel Intensity')

In [ ]:
# Reshape BOLD data and rename
print('mask list:', mask_list)
print('')
bold_data=[]
familiarization_bold_data = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    print(this_mask)
    # transpose bold data to make it timepoints x n_voxels
    bold_data = np.transpose(masked_data_all[mask_counter])
    print('Original BOLD data shape:', masked_data_all[mask_counter].shape)
    print('Transposed BOLD data shape:', bold_data.shape)
    print('')

    familiarization_bold_data[mask_counter] = bold_data

## Reward runs

In [ ]:
task = 'reward'
ses = 'ses-02'
task_index = run_names.index(task)
n_runs_reward = n_runs[task_index]
TRs_run_reward=TRs_run[task_index]-n_trunc_beginning-n_trunc_end
print('TASK:', task)
print('task index:', task_index)
print('number of task runs:', n_runs_reward)
print('TR = %s seconds' % (svd_TR))
print('TRs per run before trimming = %s' % (TRs_run[task_index]))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('TRs per run after trimming = %s' % (TRs_run_reward))
print('expected length of BOLD data series:', TRs_run_reward*n_runs_reward)
print('')

### Load reward regressor file
'/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/task-reward_regRow_all.mat'

In [ ]:
reward_row_labels={'cond': 0, 'repetition': 1, 'pairID_orig': 2, 'ipi': 3, 'order': 4, 'subcategory': 5,
                'category': 6, 'imgID': 7, 'learningTrialOnset': 8, 'trialNum': 9, 'learningBlk': 10,
                'index': 11, 'pairtype': 12, 'cycle': 13, 'rewardcond': 14, 'time': 15,
                'response': 16, 'rt': 17, 'accuracy': 18, 'onset': 19, 'recorded': 20,
                'fMRItrigger': 21, 'run': 22, 'uniquePairID': 23, 'exposureNum': 24, 'tr_original': 25, 
                'tr_trimmed': 26}
print(reward_row_labels)

In [ ]:
# load stimulus labels from regressor file for each run and concatenate
# NOTE: Regressor files are already trimmed (beginning only), but not shifted, in Matlab using gen_reward_regressor_0101.m

stim_label = []
stim_label_allruns = []
stim_label_allruns_shifted = []

# take individual run regressor files and concatenate them
for run in range(1, n_runs_reward + 1):
    in_file = (behavioral_dir + '%s_%s_task-%s_regressor-noshift-trim%dTRs_run-0%d.mat' % (sub, ses, task, n_trunc_beginning, run))
    
    # Load in data from matlab and trim end of labels
    stim_label = scipy.io.loadmat(in_file);
    stim_label = np.array(stim_label['regressor']);
    stim_label = stim_label[:,:-n_trunc_end] # trim label end

    # Store the data
    if run == 1:
        stim_label_allruns = stim_label;
    else:       
        stim_label_allruns = np.hstack((stim_label_allruns, stim_label))

print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('')
print('Trimmed (but not shifted) labels should begin with X-0-0-0-X and end with 11 trailing zeros')
print(stim_label_allruns[0,:21])
print(stim_label_allruns[0,-20:])

# Plot the labels
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label_allruns[0,:], c='orange')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')

In [ ]:
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label_allruns[reward_row_labels['run'],:], c='gray')
ax.set_ylabel('Reward run #')
ax.set_xlabel('TR')

f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label_allruns[reward_row_labels['exposureNum'],:], c='orange')
ax.set_ylabel('Exposure Num')
ax.set_xlabel('TR')

In [ ]:
# Shift the data labels to account for hemodynamic lag
shift_size = int(svd_hrf_lag / svd_TR)  # Convert the shift into TRs
print('shift by %s TRs' % (shift_size))

zero_shift = np.zeros((np.shape(stim_label_allruns)[0],shift_size)) #columns to insert at beginning
end_trim = np.shape(stim_label_allruns)[1]-shift_size #trim columns at the end

# insert shift columns at beginning and trim columns at end
stim_label_allruns_shifted = np.hstack((zero_shift,stim_label_allruns[:,0:end_trim])) 

print('stim_label_allruns has shape: ', np.shape(stim_label_allruns))
print('stim_label_allruns_shifted has shape: ', np.shape(stim_label_allruns_shifted))
print('')
print('Trimmed AND shifted labels should have 3 leading zeros and 8 trailing zeros')
print(stim_label_allruns_shifted[0,:15])
print(stim_label_allruns_shifted[0,-15:])

# Plot the original and shifted labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label_allruns[0,:], label='original', c='orange')
ax.plot(stim_label_allruns_shifted[0,:], label='shifted', c='blue')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')
ax.legend()

In [ ]:
# reset index row
index_list = np.arange(len(stim_label_allruns_shifted[1]))
stim_label_allruns_shifted[reward_row_labels['index'],:]=index_list

In [ ]:
# rename labels
reward_labels_shifted = stim_label_allruns_shifted
print('reward label list - shape:', reward_labels_shifted.shape)

### Load Reward voxel x TR BOLD data:

In [ ]:
# load voxel x TR data for each ROI
masked_data = []
masked_data_all = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    # load the mask for the corresponding ROI
    this_mask = mask_list[mask_counter]
    # Load in data from matlab
    in_file = (epi_dir + '%s_task-%s_run-ALL_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, task, n_trunc_beginning, n_trunc_end, this_mask))
    masked_data = scipy.io.loadmat(in_file)
    masked_data = np.array(masked_data['data'])
    assert masked_data.shape[1] == reward_labels_shifted.shape[1] #check that BOLD data and labels have same number 
    print(this_mask, masked_data.shape)
    masked_data_all[mask_counter] = masked_data

In [ ]:
# check dimensionality of the data and plot value of voxel_id across timeseries; make sure data are z-scored 
num_voxels = [0] * len(mask_list)
voxel_id = 10
for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    num_voxels[mask_counter] = masked_data_all[mask_counter].shape[0] #save number of voxels in each mask
    
    f, ax = plt.subplots(1,1, figsize=(14,5))
    ax.plot(masked_data_all[mask_counter][voxel_id,:])
    ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (this_mask, voxel_id))
    ax.set_xlabel('TR')
    ax.set_ylabel('Voxel Intensity')

In [ ]:
# Reshape BOLD data and rename
bold_data=[]
reward_bold_data = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    print(this_mask)
    # transpose bold data to make it timepoints x n_voxels
    bold_data = np.transpose(masked_data_all[mask_counter])
    print('Original BOLD data shape:', masked_data_all[mask_counter].shape)
    print('Transposed BOLD data shape:', bold_data.shape)
    print('')

    reward_bold_data[mask_counter] = bold_data

## Postfaces run

In [ ]:
task = 'postFaces'
ses = 'ses-02'
task_index = run_names.index(task)
n_runs_postfaces = n_runs[task_index]
TRs_run_postfaces=TRs_run[task_index]-n_trunc_beginning-n_trunc_end
print('TASK:', task)
print('task index:', task_index)
print('number of task runs:', n_runs_postfaces)
print('TR = %s seconds' % (svd_TR))
print('TRs per run before trimming = %s' % (TRs_run[task_index]))
print('%d volumes trimmed from beginning of each run' % (n_trunc_beginning))
print('%d volumes trimmed from end of each run' % (n_trunc_end))
print('TRs per run after trimming = %s' % (TRs_run_postfaces))
print('expected length of BOLD data series:', TRs_run_postfaces*n_runs_postfaces)
print('')

### Load postfaces regressor file
/jukebox/norman/emcdevitt/studies/SVD/data/behavioral/regressor/task-postStudy_regRow_reshaped.mat

In [ ]:
postfaces_row_labels={'cond': 0, 'repetition': 1, 'pairID_orig': 2, 'ipi': 3, 'order': 4, 'subcategory': 5,
                'category': 6, 'imgID': 7, 'learningBlk': 8, 'learningTrial': 9, 'time': 10,
                'response': 11, 'rt': 12, 'accuracy': 13, 'tr_original': 14, 'tr_trimmed': 15,
                'uniquePairID': 16, 'exposureNum': 17, 'index': 18} #index row to be added
print(postfaces_row_labels)

In [ ]:
# NOTE: Regressor files are already trimmed (beginning only), but not shifted, in Matlab using gen_postfaces_regressor_0101.m
stim_label = []
stim_label_shifted = []

in_file = (behavioral_dir + '%s_%s_task-%s_regressor-noshift-trim%dTRs-reshaped_run-01.mat' % (sub, ses, task, n_trunc_beginning))
stim_label = scipy.io.loadmat(in_file) # Load in data from matlab
stim_label = np.array(stim_label['regressor'])
stim_label = stim_label[:,:-n_trunc_end] # trim label end

print(in_file)
print('stim_label has shape: ', np.shape(stim_label))
print('')
print('Trimmed (but not shifted) labels should begin with alternating 1s and 0s and end with 9 trailing zeros')
print(stim_label[0,:21])
print(stim_label[0,-20:])

# Plot the labels
f, ax = plt.subplots(1,1, figsize = (12,5))
ax.plot(stim_label[0,:], c='orange') #faces only occured in condition 1(vio)
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')

In [ ]:
# Shift the data labels to account for hemodynamic lag
shift_size = int(svd_hrf_lag / svd_TR)  # Convert the shift into TRs
print('shift by %s TRs' % (shift_size))

zero_shift = np.zeros((np.shape(stim_label)[0],shift_size)) #columns to insert at beginning
end_trim = np.shape(stim_label)[1]-shift_size #trim columns at the end
# insert shift columns at beginning and trim columns at end
stim_label_shifted = np.hstack((zero_shift,stim_label[:,0:end_trim])) 

print('stim_label has shape: ', np.shape(stim_label))
print('stim_label_shifted has shape: ', np.shape(stim_label_shifted))
print('')
print('Trimmed AND shifted labels should have 3 leading zeros and 6 trailing zeros')
print(stim_label_shifted[0,:15])
print(stim_label_shifted[0,-15:])

# Plot the original and shifted labels
f, ax = plt.subplots(1,1, figsize = (20,5))
ax.plot(stim_label[0,:], label='original', c='orange')
ax.plot(stim_label_shifted[0,:], label='shifted', c='blue')
ax.set_ylabel('Stimulus category label')
ax.set_xlabel('TR')
ax.legend()

In [ ]:
# add index row
index_list = np.arange(len(stim_label_shifted[1]))
stim_label_shifted=np.vstack([stim_label_shifted, index_list])
print(stim_label_shifted[:,:10])

In [ ]:
# rename labels
postfaces_labels_shifted = stim_label_shifted
print('label list - shape:', postfaces_labels_shifted.shape)

### Load Postfaces voxel x TR BOLD data:

In [ ]:
# load voxel x TR data for each ROI
masked_data = []
masked_data_all = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    # load the mask for the corresponding ROI
    this_mask = mask_list[mask_counter]
    # Load in data from matlab
    in_file = (epi_dir + '%s_task-postfaces_run-01_space-T1w_trim%dand%dTRs_mask-%s.mat' % (sub, n_trunc_beginning, n_trunc_end, this_mask))
    masked_data = scipy.io.loadmat(in_file)
    masked_data = np.array(masked_data['data'])
    assert masked_data.shape[1] == postfaces_labels_shifted.shape[1] #check that BOLD data and labels have same number 
    print(this_mask, masked_data.shape)
    masked_data_all[mask_counter] = masked_data

In [ ]:
# check dimensionality of the data and plot value of voxel_id across timeseries; make sure data are z-scored 
num_voxels = [0] * len(mask_list)
voxel_id = 10
for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    num_voxels[mask_counter] = masked_data_all[mask_counter].shape[0] #save number of voxels in each mask
    
    f, ax = plt.subplots(1,1, figsize=(14,5))
    ax.plot(masked_data_all[mask_counter][voxel_id,:])
    ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (this_mask, voxel_id))
    ax.set_xlabel('TR')
    ax.set_ylabel('Voxel Intensity')

In [ ]:
# Reshape BOLD data and rename
bold_data=[]
postfaces_bold_data = [0] * len(mask_list)

for mask_counter in range(len(mask_list)):
    this_mask = mask_list[mask_counter]
    print(this_mask)
    # transpose bold data to make it timepoints x n_voxels
    bold_data = np.transpose(masked_data_all[mask_counter])
    print('Original BOLD data shape:', masked_data_all[mask_counter].shape)
    print('Transposed BOLD data shape:', bold_data.shape)
    print('')

    postfaces_bold_data[mask_counter] = bold_data

In [ ]:
# double checking BOLD data
voxel_id=100
mask_counter=0

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(postscenes_bold_data[mask_counter][:,voxel_id], label='postscenes', c='orange')
ax.plot(postfaces_bold_data[mask_counter][:,voxel_id], label='postfaces', c='blue')
ax.plot(familiarization_bold_data[mask_counter][:,voxel_id], label='familiarization', c='green')
ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (mask_list[mask_counter], voxel_id))
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')
ax.legend()

f, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(study_bold_data[mask_counter][:,voxel_id], label='study', c='orange')
ax.plot(reward_bold_data[mask_counter][:,voxel_id], label='reward', c='blue')
ax.set_title('Voxel time series, mask = %s, voxel id = %d' % (mask_list[mask_counter], voxel_id))
ax.set_xlabel('TR')
ax.set_ylabel('Voxel Intensity')
ax.legend()

## Compute pattern similarity
- differentiation = corr(preA,postB)
- B prediction = corr(preB,vioX) and corr(preB,vioY), then average the two correlations
- B evidence in postfaces = corr(postB,postX) and corr(postB,postY), then average the two correlations
- B evidence during reward learning = corr(postB,familiarizationA) and corr(postB,rewardAx4)

In [ ]:
# Helper function for Fisher-transformed average
def fisher_mean(correlations, axis=None):
    return np.tanh(np.mean(np.arctanh(correlations), axis=axis)) #axis 0 for columns, 1 for rows

In [ ]:
# Setup dictionaries
image_info={'condition': [], 'learningBlk': [], 'pairID': [],
            'A_category': [], 'B_category': [], 'pairtype': [], 
            'A_imgID': [], 'B_imgID': [], 'X_imgID': [], 'Y_imgID': [],
            'preA_indx': [], 'preB_indx': [], 'vioX_indx': [], 'vioY_indx': [], 
            'postB_indx': [], 'postX_indx': [], 'postY_indx': [],
            'familiarization_indx': [], 'reward1_indx': [], 'reward2_indx': [],
            'reward3_indx': [], 'reward4_indx': []}

pairwise_data={'mask': [], 'condition': [], 'learningBlk': [], 'pairID': [], 
               'preApostB': [], 'preBvioX': [], 'preBvioY': [], 'preBavgXY': [],
               'postBpostX': [], 'postBpostY': [], 'postBavgpostXY': [], 'rewardcond': [],
               'postBfam': [], 'postBreward1': [], 'postBreward2': [], 
               'postBreward3': [], 'postBreward4': []}

In [ ]:
n_pairs=96

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]

    # loop through each pair
    for i in range(1,n_pairs+1):
        # pull out trials corresponding to this pair
        study_trials = study_labels_shifted[:,(study_labels_shifted[study_row_labels['uniquePairID'],:] == i)]
        postscene_trial = postscenes_labels_shifted[:,(postscenes_labels_shifted[postscenes_row_labels['uniquePairID'],:] == i)]
        if i < 49: #condition 1 only
            postfaces_trials = postfaces_labels_shifted[:,(postfaces_labels_shifted[postfaces_row_labels['uniquePairID'],:] == i)]
        else:
            postfaces_trials = []
        fam_trial = familiarization_labels_shifted[:,(familiarization_labels_shifted[fam_row_labels['uniquePairID'],:] == i)]
        reward_trials = reward_labels_shifted[:,(reward_labels_shifted[reward_row_labels['uniquePairID'],:] == i)]
        
        this_cond = study_trials[study_row_labels['cond'],:]
        this_blk = study_trials[study_row_labels['learningBlk'],:]
        
        # double checking that condition labels match
        result = np.all(this_cond == this_cond[0])
        if result:
            this_cond = this_cond[0] #if all values are equal, then take the first value
            assert this_cond == postscene_trial[postscenes_row_labels['cond']] #and make sure it matches postscene condition
            if this_cond == 1:
                assert this_cond == postfaces_trials[postfaces_row_labels['cond'],0] #and make sure it matches postfaces trials
                assert this_cond == postfaces_trials[postfaces_row_labels['cond'],1] #and make sure it matches postfaces trials
        else:
            print('All study condition values are not same! Check pair #:', i)
        
        # double checking that learningBlks match
        result = np.all(this_blk == this_blk[0])
        if result:
            this_blk = this_blk[0] #if all values are equal, then take the first value
            assert this_blk == postscene_trial[postscenes_row_labels['learningBlk']]
            if this_cond == 1:
                assert this_blk == postfaces_trials[postfaces_row_labels['learningBlk'],0] #and make sure it matches postscene condition
                assert this_blk == postfaces_trials[postfaces_row_labels['learningBlk'],1]
        else:
            print('All learningBlk values are not same! Check pair #:', i)
            
        # get indexes
        preA_indx = study_trials[study_row_labels['index'],(study_trials[study_row_labels['cond_orig'],:] == 3) & (study_trials[study_row_labels['order'],:] == 1)]
        preA_indx = int(preA_indx[0]) #cleanup
        preB_indx = study_trials[study_row_labels['index'],(study_trials[study_row_labels['cond_orig'],:] == 3) & (study_trials[study_row_labels['order'],:] == 2)]
        preB_indx = int(preB_indx[0]) #cleanup
        postB_indx = postscene_trial[postscenes_row_labels['index']]
        postB_indx = int(postB_indx[0]) #cleanup
        
        # get brain patterns
        preA = study_bold_data[mask][preA_indx,:]
        preB = study_bold_data[mask][preB_indx,:]
        postB = postscenes_bold_data[mask][postB_indx,:]
        
        # compute differentiation correlation
        preApostB = pearsonr(preA,postB)[0] #first value is Pearson's r, second value is p-value
        
        # compute B prediction -- violation condition only
        if this_cond == 1:
            vio_indx = study_trials[study_row_labels['index'],(study_trials[study_row_labels['category'],:] == 1)]
            vioX_indx = int(vio_indx[0]) #cleanup
            vioY_indx = int(vio_indx[1]) #cleanup
            X_img = study_labels_shifted[study_row_labels['imgID'],vioX_indx]
            Y_img = study_labels_shifted[study_row_labels['imgID'],vioY_indx]
            
            # brain patterns
            vioX = study_bold_data[mask][vioX_indx,:]
            vioY = study_bold_data[mask][vioY_indx,:]
            
            # correlations
            preBvioX = pearsonr(preB,vioX)[0]
            preBvioY = pearsonr(preB,vioY)[0]
            preBavgXY = fisher_mean([preBvioX, preBvioY], axis=None)
        else: 
            vio_indx = []
            vioX_indx = np.nan
            vioY_indx = np.nan
            X_img = np.nan
            Y_img = np.nan
            vioX = []
            vioY = []
            preBvioX = np.nan
            preBvioY = np.nan
            preBavgXY = np.nan

        # compute B evidence during postfaces -- violation condition only
        if this_cond == 1:
            postX_indx = postfaces_trials[postfaces_row_labels['index'],(postfaces_trials[postfaces_row_labels['repetition'],:] == 4)] #X=4, Y=6
            postY_indx = postfaces_trials[postfaces_row_labels['index'],(postfaces_trials[postfaces_row_labels['repetition'],:] == 6)]
            postX_indx = int(postX_indx[0]) #cleanup
            postY_indx = int(postY_indx[0]) #cleanup
            postX_img = postfaces_labels_shifted[postfaces_row_labels['imgID'],postX_indx]
            postY_img = postfaces_labels_shifted[postfaces_row_labels['imgID'],postY_indx]
            assert postX_img == X_img #check that imgIDs match for vioX and postX
            assert postY_img == Y_img #check that imgIDs match for vioY and postY
            
            # brain patterns
            postX = postfaces_bold_data[mask][postX_indx,:]
            postY = postfaces_bold_data[mask][postY_indx,:]
            
            # correlations
            postBpostX = pearsonr(postB,postX)[0]
            postBpostY = pearsonr(postB,postY)[0]
            postBavgpostXY = fisher_mean([postBpostX, postBpostY], axis=None)

        else: 
            postX_indx = np.nan
            postY_indx = np.nan
            postX_img = np.nan
            postY_img = np.nan
            postX = []
            postY = []
            postBpostX = np.nan
            postBpostY = np.nan
            postBavgpostXY = np.nan
        
        # double checking reward trials: condition
        result = np.all(reward_trials[reward_row_labels['cond']] == reward_trials[reward_row_labels['cond'],0])
        if result:
            assert this_cond == reward_trials[reward_row_labels['cond'],0] #then make sure it matches study condition
        else:
            print('All condition values are not same! Check pair #:', i)
        
        # check that all rewardconds are the same
        result = np.all(reward_trials[reward_row_labels['rewardcond']] == reward_trials[reward_row_labels['rewardcond'],0])
        if result:
            rewardcond = reward_trials[reward_row_labels['rewardcond'],0]
            assert rewardcond == fam_trial[fam_row_labels['rewardcond']]
        else:
            print('All reward condition values are not same! Check pair #:', i)
        
        # check A scene image during reward trials
        result = np.all(reward_trials[reward_row_labels['imgID']] == reward_trials[reward_row_labels['imgID'],0]) #check all reward images are the same
        if result:
            assert reward_trials[reward_row_labels['imgID'],0] == study_labels_shifted[study_row_labels['imgID'],preA_indx] #check that reward matches study phase image
        else:
            print('All reward condition values are not same! Check pair #:', i)
        
        #reward trials
        familiarization_indx = int(fam_trial[fam_row_labels['index'],0])
        reward1_indx = int(reward_trials[reward_row_labels['index'],0])
        reward2_indx = int(reward_trials[reward_row_labels['index'],1])
        reward3_indx = int(reward_trials[reward_row_labels['index'],2])
        reward4_indx = int(reward_trials[reward_row_labels['index'],3])
        
        # brain patterns
        familiarization = familiarization_bold_data[mask][familiarization_indx,:]
        reward1 = reward_bold_data[mask][reward1_indx,:]
        reward2 = reward_bold_data[mask][reward2_indx,:]
        reward3 = reward_bold_data[mask][reward3_indx,:]
        reward4 = reward_bold_data[mask][reward4_indx,:]
            
        # correlations
        postBfam = pearsonr(postB,familiarization)[0]
        postBreward1 = pearsonr(postB,reward1)[0]
        postBreward2 = pearsonr(postB,reward2)[0]
        postBreward3 = pearsonr(postB,reward3)[0]
        postBreward4 = pearsonr(postB,reward4)[0]
        
        # other information
        # category info -- indoor(1) or outdoor(2)
        A_cat = study_labels_shifted[study_row_labels['subcategory'],preA_indx]
        B_cat = study_labels_shifted[study_row_labels['subcategory'],preB_indx]
        pairtype = reward_labels_shifted[reward_row_labels['pairtype'],reward1_indx]
        
        # fill in image_info for this subject (only need to do this for one loop)
        if mask == 0:
            image_info['condition'].append(this_cond)
            image_info['learningBlk'].append(this_blk)
            image_info['pairID'].append(i)
            image_info['A_category'].append(study_labels_shifted[study_row_labels['subcategory'],preA_indx])
            image_info['B_category'].append(study_labels_shifted[study_row_labels['subcategory'],preB_indx])
            image_info['pairtype'].append(pairtype)
            image_info['A_imgID'].append(study_labels_shifted[study_row_labels['imgID'],preA_indx])
            image_info['B_imgID'].append(study_labels_shifted[study_row_labels['imgID'],preB_indx])
            image_info['X_imgID'].append(X_img)
            image_info['Y_imgID'].append(Y_img)
            image_info['preA_indx'].append(preA_indx)
            image_info['preB_indx'].append(preB_indx)
            image_info['vioX_indx'].append(vioX_indx)
            image_info['vioY_indx'].append(vioY_indx)
            image_info['postB_indx'].append(postB_indx)
            image_info['postX_indx'].append(postX_indx)
            image_info['postY_indx'].append(postY_indx)
            image_info['familiarization_indx'].append(familiarization_indx)
            image_info['reward1_indx'].append(reward1_indx)
            image_info['reward2_indx'].append(reward2_indx)
            image_info['reward3_indx'].append(reward3_indx)
            image_info['reward4_indx'].append(reward4_indx)
        
        to_do = 'TODO'
        
        # fill in pairwise_data for this subject and mask
        pairwise_data['mask'].append(this_mask)
        pairwise_data['condition'].append(this_cond)
        pairwise_data['learningBlk'].append(this_blk)
        pairwise_data['pairID'].append(i)
        pairwise_data['preApostB'].append(preApostB)
        pairwise_data['preBvioX'].append(preBvioX)
        pairwise_data['preBvioY'].append(preBvioY)
        pairwise_data['preBavgXY'].append(preBavgXY)
        pairwise_data['postBpostX'].append(postBpostX)
        pairwise_data['postBpostY'].append(postBpostY)
        pairwise_data['postBavgpostXY'].append(postBavgpostXY)
        pairwise_data['rewardcond'].append(rewardcond)
        pairwise_data['postBfam'].append(postBfam)
        pairwise_data['postBreward1'].append(postBreward1)
        pairwise_data['postBreward2'].append(postBreward2)
        pairwise_data['postBreward3'].append(postBreward3)
        pairwise_data['postBreward4'].append(postBreward4)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
df = pd.DataFrame.from_dict(image_info)

# recode condition, pairtype
print(conditions)
df = df.replace({"condition": conditions})
df = df.replace({"pairtype": pair_type})

df

In [ ]:
# dataframe with pairwise pearson r values
pearson_r = pd.DataFrame.from_dict(pairwise_data)
assert pearson_r.shape[0] == len(mask_list)*n_pairs #check

# recode condition and rewardcond
pearson_r = pearson_r.replace({"condition": conditions})
pearson_r = pearson_r.replace({"rewardcond": reward_cond})
pearson_r

In [ ]:
# dataframe with pairwise fisher z values
fisher_z = pd.DataFrame.from_dict(pairwise_data)
fisher_z['preApostB'] = fisher_z['preApostB'].apply(np.arctanh) # fisher transform
fisher_z['preBvioX'] = fisher_z['preBvioX'].apply(np.arctanh)
fisher_z['preBvioY'] = fisher_z['preBvioY'].apply(np.arctanh)
fisher_z['preBavgXY'] = fisher_z['preBavgXY'].apply(np.arctanh)
fisher_z['postBpostX'] = fisher_z['postBpostX'].apply(np.arctanh)
fisher_z['postBpostY'] = fisher_z['postBpostY'].apply(np.arctanh)
fisher_z['postBavgpostXY'] = fisher_z['postBavgpostXY'].apply(np.arctanh)
fisher_z['postBfam'] = fisher_z['postBfam'].apply(np.arctanh)
fisher_z['postBreward1'] = fisher_z['postBreward1'].apply(np.arctanh)
fisher_z['postBreward2'] = fisher_z['postBreward2'].apply(np.arctanh)
fisher_z['postBreward3'] = fisher_z['postBreward3'].apply(np.arctanh)
fisher_z['postBreward4'] = fisher_z['postBreward4'].apply(np.arctanh)

# recode condition and rewardcond
fisher_z = fisher_z.replace({"condition": conditions})
fisher_z = fisher_z.replace({"rewardcond": reward_cond})

fisher_z

In [ ]:
# compute avg differentiation for each condition using fisher z values
differentiation = fisher_z.groupby(['mask', 'condition'], as_index=False)['preApostB'].mean()
assert differentiation.shape[0] == len(mask_list)*2 #2 conditions
differentiation.rename(columns={'preApostB': 'preApostB_z'}, inplace=True)

differentiation

In [ ]:
# compute vio-nonvio using Fisher z values
n_conditions = 2

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    vio = differentiation[(differentiation['mask']==this_mask) & (differentiation['condition'] == 'vio')]
    nonvio = differentiation[(differentiation['mask']==this_mask) & (differentiation['condition'] == 'nonvio')]
    vio_nonvio=[]
    vio_nonvio = vio['preApostB_z'].values[0] - nonvio['preApostB_z'].values[0]
    differentiation = differentiation.append({'mask': this_mask, 'condition': 'vio-nonvio', 'preApostB_z': vio_nonvio}, ignore_index=True)

# add column for Pearson r values and transform from z to r
differentiation['preApostB_r'] = differentiation['preApostB_z'].apply(np.tanh)

differentiation

In [ ]:
# compute avg Bprediction during familiarization and reward learning trials for each condition
Bprediction_reward = fisher_z.groupby(['mask', 'condition'], as_index=False)['postBfam', 'postBreward1', 'postBreward2', 'postBreward3', 'postBreward4'].mean()

assert Bprediction_reward.shape[0] == len(mask_list)*2 #2 conditions
Bprediction_reward.rename(columns={'postBfam': 'postBfam_z', 'postBreward1': 'postBreward1_z', 
                                   'postBreward2': 'postBreward2_z', 'postBreward3': 'postBreward3_z', 
                                   'postBreward4': 'postBreward4_z'}, inplace=True)

Bprediction_reward

In [ ]:
# compute vio-nonvio using Fisher z values
n_conditions = 2

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    vio = Bprediction_reward[(Bprediction_reward['mask']==this_mask) & (Bprediction_reward['condition'] == 'vio')]
    nonvio = Bprediction_reward[(Bprediction_reward['mask']==this_mask) & (Bprediction_reward['condition'] == 'nonvio')]
    
    vio_nonvio_fam=[]
    vio_nonvio_1=[]
    vio_nonvio_2=[]
    vio_nonvio_3=[]
    vio_nonvio_4=[]
    
    vio_nonvio_fam = vio['postBfam_z'].values[0] - nonvio['postBfam_z'].values[0]
    vio_nonvio_1 = vio['postBreward1_z'].values[0] - nonvio['postBreward1_z'].values[0]
    vio_nonvio_2 = vio['postBreward2_z'].values[0] - nonvio['postBreward2_z'].values[0]
    vio_nonvio_3 = vio['postBreward3_z'].values[0] - nonvio['postBreward3_z'].values[0]
    vio_nonvio_4 = vio['postBreward4_z'].values[0] - nonvio['postBreward4_z'].values[0]
    Bprediction_reward = Bprediction_reward.append({'mask': this_mask, 'condition': 'vio-nonvio', 'postBfam_z': vio_nonvio_fam, 
                                                    'postBreward1_z': vio_nonvio_1, 'postBreward2_z': vio_nonvio_2,
                                                    'postBreward3_z': vio_nonvio_3, 'postBreward4_z': vio_nonvio_4}, ignore_index=True)

# add column for Pearson r values and transform from z to r
Bprediction_reward['postBfam_r'] = Bprediction_reward['postBfam_z'].apply(np.tanh)
Bprediction_reward['postBreward1_r'] = Bprediction_reward['postBreward1_z'].apply(np.tanh)
Bprediction_reward['postBreward2_r'] = Bprediction_reward['postBreward2_z'].apply(np.tanh)
Bprediction_reward['postBreward3_r'] = Bprediction_reward['postBreward3_z'].apply(np.tanh)
Bprediction_reward['postBreward4_r'] = Bprediction_reward['postBreward4_z'].apply(np.tanh)


Bprediction_reward

## Compute 2nd order correlations
- Correlate B prediction (preB,vioXY) with differentiation (preA,postB) for violation pairs
- Correlate B evidence during postfaces (postB,avgpostXY) with differentiation (preA,postB) in violation condition
- Correlate B prediction during reward learning with differentiation (preA,postB)

In [ ]:
# correlate B prediction during violation events with differentiation using pairwise pearson r values
Bpredict_diff_corr={'mask': [], 'condition': [], 'BpredictDiff_z': [], 'BpredictDiff_r': []}

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    vio_only = pearson_r[(pearson_r['mask']==this_mask) & (pearson_r['condition'] == 'vio')]
    correlation = vio_only['preApostB'].corr(vio_only['preBavgXY'])
    
    Bpredict_diff_corr['mask'].append(this_mask)
    Bpredict_diff_corr['condition'].append('vio') #only computed for violation condition
    Bpredict_diff_corr['BpredictDiff_r'].append(correlation)
    Bpredict_diff_corr['BpredictDiff_z'].append(np.arctanh(correlation))

df2 = pd.DataFrame.from_dict(Bpredict_diff_corr)
df2

In [ ]:
# correlate B evidence during postfaces with differentiation using pairwise pearson r values
Bevidence_postfaces_corr={'mask': [], 'condition': [], 'BevidenceFaces_z': [], 'BevidenceFaces_r': []}

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    vio_only = pearson_r[(pearson_r['mask']==this_mask) & (pearson_r['condition'] == 'vio')]
    correlation = vio_only['preApostB'].corr(vio_only['postBavgpostXY'])
    
    Bevidence_postfaces_corr['mask'].append(this_mask)
    Bevidence_postfaces_corr['condition'].append('vio') #only computed for violation condition
    Bevidence_postfaces_corr['BevidenceFaces_r'].append(correlation)
    Bevidence_postfaces_corr['BevidenceFaces_z'].append(np.arctanh(correlation))

df3 = pd.DataFrame.from_dict(Bevidence_postfaces_corr)
df3

In [ ]:
# correlate B prediction during reward learning with differentiation using pairwise pearson r values
Bpredict_reward_diff_corr={'mask': [], 'condition': [], 'BpredictFamiliarization_diff_z': [], 
                      'BpredictReward1_diff_z': [], 'BpredictReward2_diff_z': [], 
                      'BpredictReward3_diff_z': [], 'BpredictReward4_diff_z': [],
                      'BpredictFamiliarization_diff_r': [], 'BpredictReward1_diff_r': [],
                      'BpredictReward2_diff_r': [], 'BpredictReward3_diff_r': [],
                      'BpredictReward4_diff_r': []}

corr_conditions = ['vio','nonvio','all']
# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    # loop through each condition (vio, nonvio, and both conditions together i.e. all)
    for cond in range(len(corr_conditions)):
        this_cond = corr_conditions[cond]

        if this_cond == 'all':
            selected_data = pearson_r[(pearson_r['mask']==this_mask)]
        else:
            selected_data = pearson_r[(pearson_r['mask']==this_mask) & (pearson_r['condition'] == this_cond)]
        
        correlation = selected_data['preApostB'].corr(selected_data['postBfam'])
        correlation1 = selected_data['preApostB'].corr(selected_data['postBreward1'])
        correlation2 = selected_data['preApostB'].corr(selected_data['postBreward2'])
        correlation3 = selected_data['preApostB'].corr(selected_data['postBreward3'])
        correlation4 = selected_data['preApostB'].corr(selected_data['postBreward4'])
    
        Bpredict_reward_diff_corr['mask'].append(this_mask)
        Bpredict_reward_diff_corr['condition'].append(this_cond) #only computed for violation condition
        Bpredict_reward_diff_corr['BpredictFamiliarization_diff_r'].append(correlation)
        Bpredict_reward_diff_corr['BpredictFamiliarization_diff_z'].append(np.arctanh(correlation))
        Bpredict_reward_diff_corr['BpredictReward1_diff_r'].append(correlation1)
        Bpredict_reward_diff_corr['BpredictReward1_diff_z'].append(np.arctanh(correlation1))
        Bpredict_reward_diff_corr['BpredictReward2_diff_r'].append(correlation2)
        Bpredict_reward_diff_corr['BpredictReward2_diff_z'].append(np.arctanh(correlation2))
        Bpredict_reward_diff_corr['BpredictReward3_diff_r'].append(correlation3)
        Bpredict_reward_diff_corr['BpredictReward3_diff_z'].append(np.arctanh(correlation3))
        Bpredict_reward_diff_corr['BpredictReward4_diff_r'].append(correlation4)
        Bpredict_reward_diff_corr['BpredictReward4_diff_z'].append(np.arctanh(correlation4))

df4 = pd.DataFrame.from_dict(Bpredict_reward_diff_corr)
df4

## Randomization tests

In [ ]:
# set seed for reproducibility
rng = np.random.default_rng(seed=42)

# testing
arr = np.arange(10)
print('original:', arr)
for i in range(1,10): #shuffle 10 times
    arr1 = rng.permutation(arr)
    print('permutation #', i, ':', arr1)

##### Differentiation: preApostB
- shuffle within condition (vio, nonvio)
- hold preA constant and shuffle postBs
- compute corr(preApostBshuffled)
- Fisher transform, average within condition, compute vio-nonvio difference
- save 1000 permutation values to dictionary
- compute z-score of true difference 

In [ ]:
n_iter = 1000

# Setup dictionaries
image_info_permutation={'mask': [], 'iteration': [], 'condition': [], 'pairID': [], 
                        'A_category': [], 'Bshuffled_category': [], 
                        'A_imgID': [], 'Bshuffled_imgID': [],
                        'preA_indx': [], 'postB_indx': [], 'postBshuffled_indx': [], 
                        'preApostBshuffled_r': [], 'preApostBshuffled_z': []}

In [ ]:
# df is image_info

# pull out columns of interest for each condition
vio_pairs = df.loc[(df['condition']=='vio'),['condition', 'learningBlk', 'pairID', 'A_category', 'B_category', 
                                             'A_imgID', 'B_imgID', 'preA_indx', 'postB_indx']]
nonvio_pairs = df.loc[(df['condition']=='nonvio'),['condition', 'learningBlk', 'pairID', 'A_category', 'B_category', 
                                             'A_imgID', 'B_imgID', 'preA_indx', 'postB_indx']]
display(vio_pairs)
display(nonvio_pairs)

# convert dataframes to dictionaries
vio_pairs = vio_pairs.to_dict('list')
nonvio_pairs = nonvio_pairs.to_dict('list')

In [ ]:
n_pairs=96 

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    
    # loop through each iteration
    for iter in range(1,n_iter+1):
        if (iter/500).is_integer():
            print(this_mask, 'iteration #:', iter)
            
        vio_pairs['postBshuffled_indx'] = np.nan #clear
        nonvio_pairs['postBshuffled_indx'] = np.nan #clear

        # shuffle postBs within condition
        vio_pairs['postBshuffled_indx'] = rng.permutation(vio_pairs['postB_indx'])
        nonvio_pairs['postBshuffled_indx'] = rng.permutation(nonvio_pairs['postB_indx'])

        # loop through each pair
        for i in range(1,n_pairs+1):
            
            if i < 49: #violation
                assert vio_pairs['pairID'][i-1] == i
                this_cond = vio_pairs['condition'][i-1]
                preA_indx = vio_pairs['preA_indx'][i-1]
                postB_indx = vio_pairs['postB_indx'][i-1]
                postBshuffled_indx = vio_pairs['postBshuffled_indx'][i-1]
                A_cat = vio_pairs['A_category'][i-1]
                A_img = vio_pairs['A_imgID'][i-1]
            else: #nonviolation
                assert nonvio_pairs['pairID'][i-49] == i
                this_cond = nonvio_pairs['condition'][i-49]
                preA_indx = nonvio_pairs['preA_indx'][i-49]
                postB_indx = nonvio_pairs['postB_indx'][i-49]
                postBshuffled_indx = nonvio_pairs['postBshuffled_indx'][i-49]
                A_cat = nonvio_pairs['A_category'][i-49]
                A_img = nonvio_pairs['A_imgID'][i-49]
            
            # get brain patterns
            preA = study_bold_data[mask][preA_indx,:]
            postBshuffled = postscenes_bold_data[mask][postBshuffled_indx,:]

            # compute differentiation correlation
            preApostBshuffled = pearsonr(preA,postBshuffled)[0] #first value is Pearson's r, second value is p-value
            
            #get info about postBshuffled image
            postBshuffled_cat = postscenes_labels_shifted[postscenes_row_labels['subcategory'],postBshuffled_indx]
            postBshuffled_img = postscenes_labels_shifted[postscenes_row_labels['imgID'],postBshuffled_indx]

            image_info_permutation['mask'].append(this_mask)
            image_info_permutation['iteration'].append(iter)
            image_info_permutation['condition'].append(this_cond)
            image_info_permutation['pairID'].append(i)
            image_info_permutation['A_category'].append(A_cat)
            image_info_permutation['Bshuffled_category'].append(postBshuffled_cat)
            image_info_permutation['A_imgID'].append(A_img)
            image_info_permutation['Bshuffled_imgID'].append(postBshuffled_img)
            image_info_permutation['preA_indx'].append(preA_indx)
            image_info_permutation['postB_indx'].append(postB_indx)
            image_info_permutation['postBshuffled_indx'].append(postBshuffled_indx)
            image_info_permutation['preApostBshuffled_r'].append(preApostBshuffled)
            image_info_permutation['preApostBshuffled_z'].append(np.arctanh(preApostBshuffled)) #Fisher transform

In [ ]:
# compute avg differentiation for each condition using fisher z values
differentiation_shuffled = pd.DataFrame.from_dict(image_info_permutation)
differentiation_shuffled = differentiation_shuffled.groupby(['mask', 'iteration', 'condition'], as_index=False)['preApostBshuffled_z'].mean()

assert differentiation_shuffled.shape[0] == len(mask_list)*n_iter*2 #2 conditions
differentiation_shuffled.head()

In [ ]:
# compute vio-nonvio using Fisher z values
n_conditions = 2

# setup dictionary
differentiation_perm={'mask': [], 'iteration': [], 'preApostBshuffled_vio_z': [], 
                      'preApostBshuffled_nonvio_z': [], 'preApostBshuffled_vio-nonvio_z': [], 
                      'preApostBshuffled_vio-nonvio_r': []}

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    for iter in range(1,n_iter+1):
        vio = differentiation_shuffled[(differentiation_shuffled['mask']==this_mask) & (differentiation_shuffled['iteration'] == iter) & (differentiation_shuffled['condition'] == 'vio')]
        nonvio = differentiation_shuffled[(differentiation_shuffled['mask']==this_mask) & (differentiation_shuffled['iteration'] == iter) & (differentiation_shuffled['condition'] == 'nonvio')]
        vio_nonvio=[]
        vio_nonvio = vio['preApostBshuffled_z'].values[0] - nonvio['preApostBshuffled_z'].values[0]
        differentiation_perm['mask'].append(this_mask)
        differentiation_perm['iteration'].append(iter)
        differentiation_perm['preApostBshuffled_vio_z'].append(vio['preApostBshuffled_z'].values[0])
        differentiation_perm['preApostBshuffled_nonvio_z'].append(nonvio['preApostBshuffled_z'].values[0])
        differentiation_perm['preApostBshuffled_vio-nonvio_z'].append(vio_nonvio)
        differentiation_perm['preApostBshuffled_vio-nonvio_r'].append(np.tanh(vio_nonvio))


In [ ]:
# plot histograms for each mask and compute z-score

# setup dictionary
differentiation_zscore={'mask': [], 'vio-nonvio_observed_fisherz': [], 'shuffled_mean': [],
                        'shuffled_std': [], 'vio-nonvio_permutation_zscore': []}

# convert to dataframe
shuffled_data = pd.DataFrame.from_dict(differentiation_perm)

n_subplots = len(mask_list)
f, ax = plt.subplots(1, n_subplots, sharey=True, figsize=(n_subplots*5,5)) #figsize=(10,5)
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    plot_title = 'mask=%s' %(this_mask)
    
    # get shuffled data for this mask
    select_shuffled_data = shuffled_data[(shuffled_data['mask']==this_mask)]
    assert select_shuffled_data.shape[0] == n_iter
    shuffled_mean = select_shuffled_data['preApostBshuffled_vio-nonvio_z'].mean()
    shuffled_std = select_shuffled_data['preApostBshuffled_vio-nonvio_z'].std(ddof=0) #population std
    
    # get observed values for this mask
    select_true_data = differentiation[(differentiation['mask']==this_mask) & (differentiation['condition']=='vio-nonvio')]
    observed_value = select_true_data['preApostB_z'].values[0]
    
    # compute z-score
    z_score = (observed_value - shuffled_mean)/shuffled_std
    print('z score:', z_score, 'n_iter:', n_iter)
    
    differentiation_zscore['mask'].append(this_mask)
    differentiation_zscore['vio-nonvio_observed_fisherz'].append(observed_value)
    differentiation_zscore['shuffled_mean'].append(shuffled_mean)
    differentiation_zscore['shuffled_std'].append(shuffled_std)
    differentiation_zscore['vio-nonvio_permutation_zscore'].append(z_score)
    
    sns.histplot(data=select_shuffled_data, x="preApostBshuffled_vio-nonvio_z", kde=True, ax=ax[mask])
    ax[mask].axvline(x=observed_value, color='red')
    ax[mask].set_title(plot_title)
    
df999 = pd.DataFrame.from_dict(differentiation_zscore)
display(df999)


##### B prediction: preBavgXY
- violation condition only
- hold XY and preApostB differentiation value constant; shuffle preBs
- compute corr(preBshuffled,vioX) and corr(preBshuffled,vioY); Fisher transform and average to get preBshuffled,avgXY
- correlated preBshuffled,avgXY with preApostB
- save 1000 permuted correlation values to dictionary
- compute z-score of true correlation 

In [ ]:
n_iter = 1000

In [ ]:
# df is image_info

# pull out columns of interest for each condition
vio_pairs = df.loc[(df['condition']=='vio'),['condition', 'learningBlk', 'pairID', 'A_category', 'B_category', 
                                             'preA_indx', 'preB_indx', 'vioX_indx', 'vioY_indx', 'postB_indx']]

display(vio_pairs)

# convert dataframes to dictionaries
vio_pairs = vio_pairs.to_dict('list')

In [ ]:
n_pairs=48 #violation condition only 

# Setup dictionaries
image_info_Bpredict_permutation={'mask': [], 'iteration': [], 'condition': [], 'pairID': [], 
                        'preA_indx': [], 'preB_indx': [], 'preBshuffled_indx': [], 'postB_indx': [],
                        'vioX_indx': [], 'vioY_indx': [],
                        'preBshuffledvioX_r': [], 'preBshuffledvioY_r': [], 'preBshuffledavgXY_r': [], 
                        'preApostB_r': []}

Bpredict_diff_corr_shuffled={'mask': [], 'iteration': [], 'BpredictDiff_z': [], 'BpredictDiff_r': []}

# loop through each mask
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]

    # loop through each iteration
    for iter in range(1,n_iter+1):
        if (iter/500).is_integer():
            print(this_mask, 'iteration #:', iter)
            
        # shuffle preBs within condition
        vio_pairs['preBshuffled_indx'] = np.nan #clear
        vio_pairs['preBshuffled_indx'] = rng.permutation(vio_pairs['preB_indx'])

        # loop through each pair and compute B prediction
        for i in range(1,n_pairs+1):
            assert vio_pairs['pairID'][i-1] == i
            this_cond = vio_pairs['condition'][i-1]
            preA_indx = vio_pairs['preA_indx'][i-1]
            preB_indx = vio_pairs['preB_indx'][i-1]
            postB_indx = vio_pairs['postB_indx'][i-1]
            vioX_indx = int(vio_pairs['vioX_indx'][i-1])
            vioY_indx = int(vio_pairs['vioY_indx'][i-1])
            preBshuffled_indx = vio_pairs['preBshuffled_indx'][i-1]
 
            # brain patterns
            preA = study_bold_data[mask][preA_indx,:]
            preBshuffled = study_bold_data[mask][preBshuffled_indx,:]
            vioX = study_bold_data[mask][vioX_indx,:]
            vioY = study_bold_data[mask][vioY_indx,:]
            postB = postscenes_bold_data[mask][postB_indx,:]
            
            # correlations
            preBshuffledvioX = pearsonr(preBshuffled,vioX)[0]
            preBshuffledvioY = pearsonr(preBshuffled,vioY)[0]
            preBshuffledavgXY = fisher_mean([preBshuffledvioX, preBshuffledvioY], axis=None)
            preApostB = pearsonr(preA,postB)[0]

            image_info_Bpredict_permutation['mask'].append(this_mask)
            image_info_Bpredict_permutation['iteration'].append(iter)
            image_info_Bpredict_permutation['condition'].append(this_cond)
            image_info_Bpredict_permutation['pairID'].append(i)
            image_info_Bpredict_permutation['preA_indx'].append(preA_indx)
            image_info_Bpredict_permutation['preB_indx'].append(preB_indx)
            image_info_Bpredict_permutation['preBshuffled_indx'].append(preBshuffled_indx)
            image_info_Bpredict_permutation['postB_indx'].append(postB_indx)
            image_info_Bpredict_permutation['vioX_indx'].append(vioX_indx)
            image_info_Bpredict_permutation['vioY_indx'].append(vioY_indx)
            image_info_Bpredict_permutation['preBshuffledvioX_r'].append(preBshuffledvioX)
            image_info_Bpredict_permutation['preBshuffledvioY_r'].append(preBshuffledvioY)
            image_info_Bpredict_permutation['preBshuffledavgXY_r'].append(preBshuffledavgXY)
            image_info_Bpredict_permutation['preApostB_r'].append(preApostB) 
        
        # pull out values for this iteration and correlate
        select_data = pd.DataFrame.from_dict(image_info_Bpredict_permutation)
        select_data = select_data[(select_data['mask']==this_mask) & (select_data['iteration'] == iter)]
        correlation = select_data['preApostB_r'].corr(select_data['preBshuffledavgXY_r'])
        
        Bpredict_diff_corr_shuffled['mask'].append(this_mask)
        Bpredict_diff_corr_shuffled['iteration'].append(iter)
        Bpredict_diff_corr_shuffled['BpredictDiff_r'].append(correlation)
        Bpredict_diff_corr_shuffled['BpredictDiff_z'].append(np.arctanh(correlation))

df999 = pd.DataFrame.from_dict(Bpredict_diff_corr_shuffled)
df999.head()

In [ ]:
# plot histograms for each mask and compute z-score

# setup dictionary
Bpredict_zscore={'mask': [], 'BpredictDiff_observed_fisherz': [], 'shuffled_mean': [],
                 'shuffled_std': [], 'BpredictDiff_permutation_zscore': []}

# convert to dataframe
shuffled_data = pd.DataFrame.from_dict(Bpredict_diff_corr_shuffled)

n_subplots = len(mask_list)
f, ax = plt.subplots(1, n_subplots, sharey=True, figsize=(n_subplots*5,5)) #figsize=(10,5)
for mask in range(len(mask_list)):
    this_mask = mask_list[mask]
    plot_title = 'mask=%s' %(this_mask)
    
    # get shuffled data for this mask
    select_shuffled_data = shuffled_data[(shuffled_data['mask']==this_mask)]
    assert select_shuffled_data.shape[0] == n_iter
    shuffled_mean = select_shuffled_data['BpredictDiff_z'].mean()
    shuffled_std = select_shuffled_data['BpredictDiff_z'].std(ddof=0) #population std
    
    # get observed values for this mask
    df2 = pd.DataFrame.from_dict(Bpredict_diff_corr)
    select_true_data = df2[(df2['mask']==this_mask)]
    observed_value = select_true_data['BpredictDiff_z'].values[0]
    
    # compute z-score
    z_score = (observed_value - shuffled_mean)/shuffled_std
    print('observed_value:', observed_value, 'z score:', z_score, 'n_iter:', n_iter)
    
    Bpredict_zscore['mask'].append(this_mask)
    Bpredict_zscore['BpredictDiff_observed_fisherz'].append(observed_value)
    Bpredict_zscore['shuffled_mean'].append(shuffled_mean)
    Bpredict_zscore['shuffled_std'].append(shuffled_std)
    Bpredict_zscore['BpredictDiff_permutation_zscore'].append(z_score)
    
    sns.histplot(data=select_shuffled_data, x="BpredictDiff_z", kde=True, ax=ax[mask], alpha=0.25)
    ax[mask].axvline(x=observed_value, color='red')
    ax[mask].set_title(plot_title)


## Save subject-specific file

In [ ]:
# convert differentiation dataframe to dictionary before saving
avg_differentiation = differentiation.to_dict('list')
avg_Bprediction_reward = Bprediction_reward.to_dict('list')

# save dictionaries to .npy file
outfile = out_dir + '%s_pattern_similarity_analyses' % (sub)
print('saving to file: ', outfile)
print('')
np.savez(outfile, image_info=image_info, pairwise_data=pairwise_data, 
         differentiation=avg_differentiation, Bprediction_reward=avg_Bprediction_reward,
         Bprediction_diff=Bpredict_diff_corr, Bevidence_faces=Bevidence_postfaces_corr, 
         Bprediction_reward_diff=Bpredict_reward_diff_corr, image_info_differentiation_permutation=image_info_permutation,
         differentiation_shuffled_values=differentiation_perm, differentiation_zscores=differentiation_zscore, 
         image_info_Bpredict_permutation=image_info_Bpredict_permutation, BpredictDiff_corr_shuffled_values=Bpredict_diff_corr_shuffled,
         BpredictDiff_corr_zscores=Bpredict_zscore)
print('save complete')

In [ ]:
# # example of how to load saved npz data
# infile = out_dir + '/%s_differentiation_prediction.npz' % (sub)
# in_data = np.load(infile, allow_pickle=True)
# print('loaded dictionaries:', in_data.files)
# diff = in_data['differentiation'].item()
# example = pd.DataFrame.from_dict(diff)
# example